# Calibrating the $X$ gate

Topic: Introduction to superconducting qubits and quantum computers

$\newcommand{\ket}[1]{\lvert #1 \rangle}$

We will build an implementation of the $X$ gate from scratch using the `pulse` subpackage of Qiskit on a simulator and a IBM Quantum backend.

In [ ]:
from collections.abc import Callable
import pprint
from typing import Optional
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from qiskit import QuantumCircuit, pulse, transpile
from qiskit.circuit import Gate, Parameter
from qiskit.primitives import BackendSampler
from qiskit.providers import Backend
from qiskit.qobj.utils import MeasLevel
from qiskit_experiments.data_processing import BaseDiscriminator
from qiskit_experiments.test import SingleTransmonTestBackend
from qiskit_ibm_runtime import Options, QiskitRuntimeService, Sampler, Session
from qiskit_ibm_runtime.ibm_qubit_properties import IBMQubitProperties

## Pulse examples

To control the qubits with microwave pulses, we build pulse schedules and attach them to circuit objects. The standard method for building a pulse schedule is through the `pulse.build()` function.

In [ ]:
# Builds a ScheduleBlock object named sched
with pulse.build(name='test_schedule') as sched:
    # "Play" instruction consists of a pulse shape and the channel to play it on
    pulse.play(pulse.Gaussian(duration=160, amp=0.1, sigma=40, name='Gaus'),
               pulse.DriveChannel(0))
    pulse.play(pulse.GaussianSquare(duration=512, amp=0.2, sigma=64, width=256, name='GausSq'),
               pulse.ControlChannel(2))
    # There also exist other instructions such as pulse phase shifting and delay:
    pulse.shift_phase(-np.pi / 4., pulse.DriveChannel(0))
    pulse.delay(320, pulse.DriveChannel(0))
    pulse.play(pulse.Gaussian(duration=160, amp=0.1, sigma=40, name='Gaus'),
               pulse.DriveChannel(0))

# Visualize the schedule with `draw()`
sched.draw()

Pulse schedules is attached to quantum circuits as "calibrations" of custom gates.

In [ ]:
from qiskit.circuit.library import XGate

circuit = QuantumCircuit(2)
# This is equivalent to circuit.x(0)
circuit.append(XGate(), [0])
# Appending a custom gate
circuit.append(Gate('test_gate', 1, []), [1])

# Then bind the schedule to the gate
# add_calibration(gate_name, physical_qubits, schedule)
circuit.add_calibration('test_gate', [3], sched)

# Schedules are not visible in the circuit visualization directly
circuit.draw('mpl')

In [ ]:
# You can find the attached calibrations in circuit.calibrations
# Structure:
#{
# gate_name: {
#             (qubits, parameters): schedule
#            }
#}
print(circuit.calibrations)
circuit.calibrations['test_gate'][((3,), ())].draw()

## Utility functions

In [ ]:
def execute(
    circuits: list[QuantumCircuit],
    physical_qubit: int,
    backend: Optional[Backend] = None,
    session: Optional[Session] = None,
    shots: int = 10000,
    tag: Optional[str] = None
) -> np.ndarray:
    """Run single-qubit circuits on a Sampler and return the quasi-probability of obtaining '1'."""
    circuits = transpile(circuits, backend=backend, initial_layout=[physical_qubit], optimization_level=0)
    runtime_options = {'shots': shots, 'meas_level': MeasLevel.CLASSIFIED}
    if session is not None:
        # Don't transpile at the backend but apply readout error mitigation
        options = Options(optimization_level=0, resilience_level=1, transpilation={'skip_transpilation': True})
        sampler = Sampler(session=session, options=options)
        if tag:
            runtime_options['job_tags'] = [tag]
    else:
        sampler = BackendSampler(backend)

    result = sampler.run(circuits, **runtime_options).result()
    return np.array([d.get(1, 0.) for d in result.quasi_dists])

In [ ]:
def fit(
    curve: Callable,
    xval: np.ndarray,
    yval: np.ndarray,
    p0: tuple[float, ...],
    shots: Optional[int] = None
) -> np.ndarray:
    """Run curve_fit and plot the results."""
    popt, _ = curve_fit(curve, xval, yval, p0=p0)

    _, ax = plt.subplots()
    if shots is None:
        ax.scatter(xval, yval, markersize=3)
    else:
        ax.errorbar(xval, yval, yerr=np.sqrt(yval * (1. - yval) / shots), fmt='o', markersize=3)
    xfine = np.linspace(xval[0], xval[-1], 400)
    ax.plot(xfine, curve(xfine, *popt))

    return popt

## Calibration routines

### Spectroscopy

In [ ]:
def rough_frequency_calibration(
    backend: Backend,
    physical_qubit: int,
    shots: int = 10000,
    session: Optional[Session] = None
) -> float:
    """Weakly drive the qubit at different frequencies to identify the resonance."""
    # Build a parametrized pulse schedule
    freq_offset = Parameter('freq_offset')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='spectrocopy') as spectroscopy_sched:
        with pulse.frequency_offset(freq_offset, drive_channel):
            pulse.play(pulse.Gaussian(duration=512, amp=0.01, sigma=128, name='Probe'), drive_channel)

    # Build a parametrized circuit using the schedule
    circuit = QuantumCircuit(1, 1)
    # Custom gate has one free parameter
    circuit.append(Gate('spectroscopy', 1, [freq_offset]), [0])
    circuit.measure(0, 0)
    # Attach the schedule specifying freq_offset as the gate parameter
    circuit.add_calibration('spectroscopy', [physical_qubit], spectroscopy_sched, params=[freq_offset])

    qubit_props = backend.qubit_properties(physical_qubit)

    # Scan frequencies relative to the drive channel frequency (qubit resonance)
    freq_offsets = np.linspace(-1.e+8, 1.e+8, 40)

    # Instantiate concrete circuits using assign_parameters()
    circuits = [
        circuit.assign_parameters({freq_offset: offset_value}, inplace=False)
        for offset_value in freq_offsets
    ]

    # Run the sampler
    yvals = execute(circuits, physical_qubit, backend=backend, session=session, shots=shots, tag='rough_frequency')

    # Gaussian fit function
    def resonance_curve(x, f0, a, b, sigma):
        return a * np.exp(-(x - f0)**2 / 2. / sigma**2) + b

    # Fit parameter guesses
    ipeak = np.argmax(yvals)
    f0_guess = freq_offsets[ipeak]
    b_guess = np.amin(yvals)
    a_guess = yvals[ipeak] - b_guess
    ionesigma = np.argmin(np.abs(yvals - (a_guess * np.exp(-0.5) + b_guess)))
    sigma_guess = np.abs(freq_offsets[ionesigma] - f0_guess)

    p0 = (f0_guess, a_guess, b_guess, sigma_guess)

    popt = fit(resonance_curve, freq_offsets, yvals, p0, shots)

    return popt[0] + qubit_props.frequency

In [ ]:
class Discriminator(BaseDiscriminator):
    def __init__(self, backend):
        c0 = QuantumCircuit(1, 1)
        c0.measure(0, 0)
        c1 = QuantumCircuit(1, 1)
        c1.x(0)
        c1.measure(0, 0)
        cc = transpile([c0, c1], backend=backend)
        result = backend.run(cc, shots=10000, meas_level=1, meas_return='single').result()
        mem0 = np.squeeze(result.data(0)['memory'])
        mem1 = np.squeeze(result.data(1)['memory'])
        centers = KMeans(n_clusters=2, random_state=0).fit(np.concatenate([mem0, mem1])).cluster_centers_

        dvec = mem0[:, None, :] - centers[None, :, :]
        distmean = np.mean(np.sqrt(np.sum(np.square(dvec), axis=-1)), axis=0)
        i1 = np.argmin(distmean)
        i2 = np.argmax(distmean)
        
        # The norm vector points from i1 to i2
        i1toi2 = centers[i2] - centers[i1]
        centroids_dist = np.sqrt(np.dot(i1toi2, i1toi2))
        
        self.vnorm = i1toi2 / centroids_dist
        self.dist = np.dot(self.vnorm, np.mean(centers, axis=0))
        
    def predict(self, data):
        return np.asarray(np.dot(data, self.vnorm) - self.dist > 0.).astype(int).astype(str)

backend = SingleTransmonTestBackend(noise=False)
backend.discriminator = Discriminator(backend)
physical_qubit = 0

In [ ]:
qubit_frequency = rough_frequency_calibration(backend, physical_qubit)

In [ ]:
def rough_amplitude_calibration(
    backend: Backend,
    physical_qubit: int,
    qubit_frequency: float,
    shots: int = 10000,
    session: Optional[Session] = None
) -> tuple[float, float]:
    """Vary the amplitude of a fixed-width pulse and return the amplitudes for X and SX gates."""
    # Parametrized schedule
    amp = Parameter('amp')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='rabi') as rabi_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=amp, sigma=40, name='Probe'), drive_channel)

    # Parametrized circuit
    circuit = QuantumCircuit(1, 1)
    circuit.append(Gate('rabi', 1, [amp]), [0])
    circuit.measure(0, 0)
    circuit.add_calibration('rabi', [physical_qubit], rabi_sched, params=[amp])

    # Amplitude values
    amplitudes = np.linspace(0., 0.2, 40)
    
    circuits = [
        circuit.assign_parameters({amp: amp_value}, inplace=False)
        for amp_value in amplitudes
    ]

    # Run the sampler
    yvals = execute(circuits, physical_qubit, backend=backend, session=session, shots=shots, tag='rough_amplitude')

    # Sinusoidal fit function
    def oscillation_curve(x, freq, phase, a, b):
        return a * np.cos(2. * np.pi * x * freq + phase) + b

    spectrum = np.fft.fft(yvals - 0.5)
    ipeak = np.argmax(spectrum[:len(amplitudes) // 2])
    # Pulse with amp=amplitudes[ipeak] effects a full cycle
    freq_guess = 2. * ipeak / amplitudes[-1]
    phase_guess = 0.
    a_guess = -0.5
    b_guess = 0.5

    p0 = (freq_guess, phase_guess, a_guess, b_guess)

    popt = fit(oscillation_curve, amplitudes, yvals, p0, shots)

    # Amplitude for X should effect a half cycle
    x_amp = 0.5 / popt[0]
    # Amplitude for SX is roughly half of X amp
    sx_amp = x_amp / 2.

    return x_amp, sx_amp

In [ ]:
x_amp, sx_amp = rough_amplitude_calibration(backend, physical_qubit, qubit_frequency)

In [ ]:
def fine_frequency_calibration(
    backend: Backend,
    physical_qubit: int,
    qubit_frequency: float,
    sx_amp: float,
    shots: int = 10000,
    session: Optional[Session] = None
) -> float:
    """Perform a fine-grained re-calibration of the resonance frequency."""
    # Parametrized schedule
    delay_unit = 32
    ndelay = Parameter('ndelay')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='ramsey') as ramsey_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=sx_amp, sigma=40), drive_channel)
        pulse.delay(ndelay * delay_unit, drive_channel)
        with pulse.phase_offset(np.pi / 2. * ndelay, drive_channel):
            pulse.play(pulse.Gaussian(duration=160, amp=sx_amp, sigma=40), drive_channel)

    # Parametrized circuit
    circuit = QuantumCircuit(1, 1)
    circuit.append(Gate('ramsey', 1, [ndelay]), [0])
    circuit.measure(0, 0)
    circuit.add_calibration('ramsey', [physical_qubit], ramsey_sched, params=[ndelay])

    ndelays = np.arange(32)
    
    circuits = [
        circuit.assign_parameters({ndelay: ndelay_val}, inplace=False)
        for ndelay_val in ndelays
    ]

    # Run the sampler
    yvals = execute(circuits, physical_qubit, backend=backend, session=session, shots=shots, tag='fine_frequency')

    # Error amplification analysis
    def error_amplification_curve(n, delta, a, b, offset):
        return a * np.cos(0.5 * np.pi * n + delta * (n + 1) + offset) + b
    
    delta_guess = 0.
    a_guess = 0.5
    b_guess = 0.5
    offset_guess = 0.

    p0 = (delta_guess, a_guess, b_guess, offset_guess)

    popt = fit(error_amplification_curve, ndelays, yvals, p0, shots)

    return qubit_frequency - popt[0] / (delay_unit * backend.dt * 2. * np.pi)

In [ ]:
refined_qubit_frequency = fine_frequency_calibration(backend, physical_qubit, qubit_frequency, sx_amp)

Did the recalibration give a more accurate frequency estimate?

In [ ]:
true_frequency = backend.qubit_properties(physical_qubit).frequency
print('Rough Δf =', qubit_frequency - true_frequency)
print('Fine Δf =', refined_qubit_frequency - true_frequency)

In [ ]:
def fine_amplitude_calibration(
    backend: Backend,
    physical_qubit: int,
    qubit_frequency: float,
    x_amp: float,
    sx_amp: float,
    shots: int = 10000,
    session: Optional[Session] = None
) -> tuple[float, float]:
    """Perform a fine-grained re-calibration of the resonance frequency."""
    # X and SX schedules
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='x') as x_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=x_amp, sigma=40), drive_channel)

    with pulse.build(name='sx') as sx_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=sx_amp, sigma=40), drive_channel)

    repetitions = np.arange(1, 25)
        
    # X calibration circuits
    x_circuits = []
    sx_circuits = []
    for nrep in repetitions:
        circuit = QuantumCircuit(1, 1)
        circuit.sx(0)
        for _ in range(nrep):
            circuit.x(0)
        circuit.measure(0, 0)
        circuit.add_calibration('x', [physical_qubit], x_sched)
        circuit.add_calibration('sx', [physical_qubit], sx_sched)
        x_circuits.append(circuit)

        circuit = QuantumCircuit(1, 1)
        for _ in range(nrep):
            circuit.sx(0)
        circuit.measure(0, 0)
        circuit.add_calibration('sx', [physical_qubit], sx_sched)
        sx_circuits.append(circuit)

    # Run the sampler
    x_yvals = execute(x_circuits, physical_qubit, backend=backend, session=session, shots=shots, tag='fine_x_amplitude')
    sx_yvals = execute(sx_circuits, physical_qubit, backend=backend, session=session, shots=shots, tag='fine_sx_amplitude')

    # Error amplification analysis
    def x_curve(n, delta, a, b, offset):
        return a * np.cos((np.pi + delta) * n + offset) + b

    def sx_curve(n, delta, a, b, offset):
        return a * np.cos((0.5 * np.pi + delta) * n + offset) + b

    delta_guess = 0.
    a_guess = 0.5
    b_guess = 0.5
    
    # X fit
    offset_guess = -0.5 * np.pi
    p0 = (delta_guess, a_guess, b_guess, offset_guess)
    x_popt = fit(x_curve, repetitions, x_yvals, p0, shots)

    # SX fit
    offset_guess = -np.pi
    p0 = (delta_guess, a_guess, b_guess, offset_guess)
    sx_popt = fit(sx_curve, repetitions, sx_yvals, p0, shots)

    new_x_amp = x_amp * np.pi / (np.pi + x_popt[0])
    # Amplitude for SX is roughly half of X amp
    new_sx_amp = sx_amp * 0.5 * np.pi / (0.5 * np.pi + sx_popt[0])

    return new_x_amp, new_sx_amp

In [ ]:
refined_x_amp, refined_sx_amp = fine_amplitude_calibration(backend, physical_qubit, qubit_frequency, x_amp, sx_amp)

Did it work?

In [ ]:
rerefined_x_amp, rerefined_sx_amp = fine_amplitude_calibration(backend, physical_qubit, qubit_frequency, refined_x_amp, refined_sx_amp)

## Using an IBM Quantum backend

In [ ]:
instance = None
# Uncomment and edit below if you have access to multiple instances and want to specify which one to use
#instance = 'hub/group/project'
service = QiskitRuntimeService(channel='ibm_quantum', instance=instance)
backend = service.least_busy(operational=True, simulator=False)

In [ ]:
with Session(service=runtime_service, backend=backend) as session:
    qubit_frequency = rough_frequency_calibration(backend, physical_qubit, session=session)
    x_amp, sx_amp = rough_amplitude_calibration(backend, physical_qubit, qubit_frequency, session=session)
    refined_qubit_frequency = fine_frequency_calibration(backend, physical_qubit, qubit_frequency, sx_amp, session=session)
    refined_x_amp, refined_sx_amp = fine_amplitude_calibration(backend, physical_qubit, qubit_frequency, x_amp, sx_amp, session=session)